In [1]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add the src directory to the path so we can import our modules
sys.path.append('/home/mg873uh/Projects_kb/baseline-codebase/src')

from data.datamodule import PretrainDataModule
from data.dataset import PretrainDatasetCombinedPatient, CombinedPretrainingV2,TrackedUniquePatientBatchSampler
from data.pretrain_split import get_pretrain_split_config
from augmentations.augmentation_composer import get_pretrain_augmentations, get_val_augmentations
from utils.utils import SimplePathConfig

print("Imports successful!")
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader

Imports successful!


In [2]:
dataset = CombinedPretrainingV2(patch_size=(96, 96, 96), data_dir= "/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/")
dataset.__getitem__(1)

{'view1': tensor([[[[4.4823e-01, 4.4537e-01, 4.4310e-01,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [4.5662e-01, 4.5208e-01, 4.4889e-01,  ..., 2.1518e-15,
            1.6004e-16, 1.0730e-15],
           [4.6529e-01, 4.6045e-01, 4.5734e-01,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [4.8109e-01, 4.8930e-01, 4.9566e-01,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [4.9147e-01, 5.0359e-01, 5.1426e-01,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [5.1310e-01, 5.3143e-01, 5.4837e-01,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00]],
 
          [[5.0020e-01, 4.9635e-01, 4.9122e-01,  ..., 1.2848e-15,
            9.5563e-17, 6.4069e-16],
           [4.9866e-01, 4.9302e-01, 4.8767e-01,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [4.9797e-01, 4.9156e-01, 4.8650e-01,  ..., 1.8092e-14,
            1.3456e-15, 9.0216e-15],
           ...,
           [4.9830e-01, 5.021

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import DataLoader

# Initialize dataset
dataset = CombinedPretrainingV2(
    patch_size=(96, 96, 96), 
    data_dir="/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/",
    mode='multimodal'
)

# Better visualization with larger images
def visualize_batch_grid(dataset, batch_size=16, figsize=(20, 20)):
    """Create a grid visualization with larger images"""
    
    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    batch = next(iter(dataloader))
    
    # Extract data
    if isinstance(batch['view1'], dict):
        view1_data = batch['view1']['image']
        view2_data = batch['view2']['image']
    else:
        view1_data = batch['view1']
        view2_data = batch['view2']
    
    actual_batch_size = view1_data.shape[0]
    
    # Create 4x8 grid (4 rows, 8 columns) for 16 pairs
    n_rows = 4
    n_cols = 8  # 4 pairs per row, each pair takes 2 columns
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    
    # Plot each pair
    pair_idx = 0
    for row in range(n_rows):
        for col in range(0, n_cols, 2):  # Step by 2 for each pair
            if pair_idx >= actual_batch_size:
                # Hide unused subplots
                axes[row, col].axis('off')
                axes[row, col+1].axis('off')
                continue
                
            # Get middle slice
            depth = view1_data.shape[2]
            mid_slice = depth // 2
            
            # Extract images
            img1 = view1_data[pair_idx, 0, mid_slice, :, :].cpu().numpy()
            img2 = view2_data[pair_idx, 0, mid_slice, :, :].cpu().numpy()
            
            # Plot view 1
            im1 = axes[row, col].imshow(img1, cmap='gray')
            axes[row, col].set_title(f"{batch['patient'][pair_idx]}\nView 1", fontsize=10)
            axes[row, col].axis('off')
            
            # Plot view 2
            im2 = axes[row, col+1].imshow(img2, cmap='gray')
            axes[row, col+1].set_title(f"{batch['patient'][pair_idx]}\nView 2", fontsize=10)
            axes[row, col+1].axis('off')
            
            pair_idx += 1
    
    plt.suptitle(f'Batch of {actual_batch_size} Image Pairs', fontsize=16)
    plt.tight_layout()
    return fig, batch

# Even better: Show fewer pairs but larger
def visualize_batch_large(dataset, n_pairs=8, figsize=(24, 12)):
    """Show fewer pairs but much larger"""
    
    # Create dataloader
    dataloader = DataLoader(dataset, batch_size=n_pairs, shuffle=True)
    batch = next(iter(dataloader))
    
    # Extract data
    if isinstance(batch['view1'], dict):
        view1_data = batch['view1']['image']
        view2_data = batch['view2']['image']
    else:
        view1_data = batch['view1']
        view2_data = batch['view2']
    
    actual_batch_size = view1_data.shape[0]
    
    # 2 rows, 4 pairs per row
    n_rows = 2
    n_cols = 8  # 4 pairs × 2 images
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    
    pair_idx = 0
    for row in range(n_rows):
        for col in range(0, n_cols, 2):
            if pair_idx >= actual_batch_size:
                axes[row, col].axis('off')
                axes[row, col+1].axis('off')
                continue
                
            # Get middle slice
            mid_slice = view1_data.shape[2] // 2
            
            # Extract images
            img1 = view1_data[pair_idx, 0, mid_slice, :, :].cpu().numpy()
            img2 = view2_data[pair_idx, 0, mid_slice, :, :].cpu().numpy()
            
            # Calculate robust min/max for better contrast
            vmin = np.percentile([img1, img2], 1)
            vmax = np.percentile([img1, img2], 99)
            
            # Plot view 1
            axes[row, col].imshow(img1, cmap='gray', vmin=vmin, vmax=vmax)
            axes[row, col].set_title(f"Patient {batch['patient'][pair_idx]} - View 1", fontsize=12)
            axes[row, col].axis('off')
            
            # Plot view 2
            axes[row, col+1].imshow(img2, cmap='gray', vmin=vmin, vmax=vmax)
            axes[row, col+1].set_title(f"View 2", fontsize=12)
            axes[row, col+1].axis('off')
            
            # Add text showing intensity range
            axes[row, col].text(0.02, 0.98, f"Range: [{img1.min():.1f}, {img1.max():.1f}]", 
                               transform=axes[row, col].transAxes, 
                               verticalalignment='top',
                               bbox=dict(boxstyle='round', facecolor='white', alpha=0.7),
                               fontsize=8)
            
            pair_idx += 1
    
    plt.suptitle(f'{actual_batch_size} Image Pairs (Middle Slices)', fontsize=18)
    plt.tight_layout()
    return fig, batch

# Interactive visualization - cycle through pairs one at a time
def visualize_pairs_interactive(dataset, n_pairs=16):
    """Show pairs one at a time in a large format"""
    
    # Get batch
    dataloader = DataLoader(dataset, batch_size=n_pairs, shuffle=True)
    batch = next(iter(dataloader))
    
    # Extract data
    if isinstance(batch['view1'], dict):
        view1_data = batch['view1']['image']
        view2_data = batch['view2']['image']
    else:
        view1_data = batch['view1']
        view2_data = batch['view2']
    
    # Show first few pairs in separate figures
    for i in range(min(4, view1_data.shape[0])):  # Show first 4 pairs
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
        
        # Get middle slice
        mid_slice = view1_data.shape[2] // 2
        
        img1 = view1_data[i, 0, mid_slice, :, :].cpu().numpy()
        img2 = view2_data[i, 0, mid_slice, :, :].cpu().numpy()
        
        # Use percentiles for better contrast
        vmin = np.percentile([img1, img2], 1)
        vmax = np.percentile([img1, img2], 99)
        
        # View 1
        im1 = ax1.imshow(img1, cmap='gray', vmin=vmin, vmax=vmax)
        ax1.set_title(f"Patient {batch['patient'][i]} - View 1\nShape: {img1.shape}", fontsize=14)
        ax1.axis('off')
        plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
        
        # View 2
        im2 = ax2.imshow(img2, cmap='gray', vmin=vmin, vmax=vmax)
        ax2.set_title(f"Patient {batch['patient'][i]} - View 2\nShape: {img2.shape}", fontsize=14)
        ax2.axis('off')
        plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
        
        plt.suptitle(f'Pair {i+1}/{view1_data.shape[0]}', fontsize=16)
        plt.tight_layout()
        plt.show()

# Create a summary view with multiple slices per pair
def visualize_batch_multislice(dataset, n_pairs=4, n_slices=5, figsize=(20, 16)):
    """Show multiple slices for each pair"""
    
    dataloader = DataLoader(dataset, batch_size=n_pairs, shuffle=True)
    batch = next(iter(dataloader))
    
    # Extract data
    if isinstance(batch['view1'], dict):
        view1_data = batch['view1']['image']
        view2_data = batch['view2']['image']
    else:
        view1_data = batch['view1']
        view2_data = batch['view2']
    
    actual_batch_size = view1_data.shape[0]
    depth = view1_data.shape[2]
    
    # Calculate slice positions
    slice_positions = np.linspace(depth * 0.2, depth * 0.8, n_slices, dtype=int)
    
    # Create subplot grid
    fig, axes = plt.subplots(actual_batch_size * 2, n_slices, figsize=figsize)
    if actual_batch_size == 1:
        axes = axes.reshape(2, -1)
    
    for pair_idx in range(actual_batch_size):
        for slice_idx, slice_pos in enumerate(slice_positions):
            # View 1
            img1 = view1_data[pair_idx, 0, slice_pos, :, :].cpu().numpy()
            ax1 = axes[pair_idx * 2, slice_idx]
            ax1.imshow(img1, cmap='gray')
            if slice_idx == 0:
                ax1.set_ylabel(f"{batch['patient'][pair_idx]}\nView 1", fontsize=10)
            if pair_idx == 0:
                ax1.set_title(f"Slice {slice_pos}", fontsize=10)
            ax1.axis('off')
            
            # View 2
            img2 = view2_data[pair_idx, 0, slice_pos, :, :].cpu().numpy()
            ax2 = axes[pair_idx * 2 + 1, slice_idx]
            ax2.imshow(img2, cmap='gray')
            if slice_idx == 0:
                ax2.set_ylabel("View 2", fontsize=10)
            ax2.axis('off')
    
    plt.suptitle(f'{actual_batch_size} Pairs × {n_slices} Slices Each', fontsize=16)
    plt.tight_layout()
    return fig, batch

# Use the functions
print("Creating large grid visualization...")
fig1, batch = visualize_batch_large(dataset, n_pairs=8)
plt.show()

print("\nShowing individual pairs...")
visualize_pairs_interactive(dataset, n_pairs=4)

print("\nCreating multi-slice visualization...")
fig2, batch = visualize_batch_multislice(dataset, n_pairs=4, n_slices=5)
plt.show()

Creating large grid visualization...


IndexError: list index out of range

In [ ]:
# Check dataset statistics
dataset = CombinedPretrainingV2(
    patch_size=(96, 96, 96), 
    data_dir="/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/",
    mode='multimodal'  # or 'augmentation'
)

# Basic statistics
print("="*50)
print("DATASET STATISTICS")
print("="*50)
print(f"Total number of samples in dataset: {len(dataset)}")
print(f"Number of unique patients: {len(dataset.patients)}")
print(f"Dataset mode: {dataset.mode}")
print()

# Detailed patient information
print("PATIENT BREAKDOWN:")
print("-"*50)
modality_counts = {}
for patient, modalities in dataset.patients.items():
    n_modalities = len(modalities)
    print(f"Patient {patient}: {n_modalities} modalities")
    
    # Track distribution
    if n_modalities not in modality_counts:
        modality_counts[n_modalities] = 0
    modality_counts[n_modalities] += 1

print("\nMODALITY DISTRIBUTION:")
print("-"*50)
for n_mod, count in sorted(modality_counts.items()):
    print(f"Patients with {n_mod} modalities: {count}")

# Calculate total possible pairs
print("\nPAIR STATISTICS:")
print("-"*50)
if dataset.mode == 'multimodal':
    total_pairs = 0
    for patient, modalities in dataset.patients.items():
        n = len(modalities)
        if n >= 2:
            # Number of unique pairs = n choose 2 = n*(n-1)/2
            pairs = n * (n - 1) // 2
            total_pairs += pairs
    print(f"Total possible unique pairs (multimodal mode): {total_pairs}")
    print(f"Actual dataset length: {len(dataset)}")
    print("Note: In multimodal mode, only patients with 2+ modalities are included")
else:
    print(f"Total samples (augmentation mode): {len(dataset)}")
    print("Note: In augmentation mode, each modality is a sample")

# Show some example file names
print("\nEXAMPLE FILES:")
print("-"*50)
# Show first 3 patients
for i, (patient, modalities) in enumerate(list(dataset.patients.items())[:3]):
    print(f"\nPatient {patient}:")
    for mod_path in modalities[:3]:  # Show up to 3 modalities per patient
        print(f"  - {os.path.basename(mod_path)}")
    if len(modalities) > 3:
        print(f"  ... and {len(modalities) - 3} more modalities")

# Check what index_to_patient contains
print("\nINDEX MAPPING CHECK:")
print("-"*50)
print(f"Length of index_to_patient: {len(dataset.index_to_patient)}")
print(f"First 5 entries:")
for i in range(min(5, len(dataset.index_to_patient))):
    patient_name, modalities = dataset.index_to_patient[i]
    print(f"  Index {i}: Patient {patient_name}, {len(modalities)} modalities available")

# If using V2, check the all_pairs
if hasattr(dataset, 'all_pairs'):
    print(f"\nCombinedPretrainingV2 - all_pairs length: {len(dataset.all_pairs)}")
    print("First 5 pairs:")
    for i in range(min(5, len(dataset.all_pairs))):
        patient, path1, path2 = dataset.all_pairs[i]
        print(f"  Pair {i}: Patient {patient}")
        print(f"    - {os.path.basename(path1)}")
        print(f"    - {os.path.basename(path2)}")

# Test accessing some samples
print("\nTEST ACCESS:")
print("-"*50)
try:
    sample = dataset[0]
    print(f"Successfully accessed sample 0")
    print(f"Sample keys: {sample.keys()}")
    if isinstance(sample['view1'], dict):
        print(f"View1 shape: {sample['view1']['image'].shape}")
    else:
        print(f"View1 shape: {sample['view1'].shape}")
except Exception as e:
    print(f"Error accessing sample: {e}")

# Summary
print("\n" + "="*50)
print("SUMMARY")
print("="*50)
total_files = sum(len(mods) for mods in dataset.patients.values())
print(f"Total .npy files found: {total_files}")
print(f"Dataset length (accessible samples): {len(dataset)}")
if dataset.mode == 'multimodal':
    excluded_patients = sum(1 for mods in dataset.patients.values() if len(mods) < 2)
    print(f"Patients excluded (less than 2 modalities): {excluded_patients}")

DATASET STATISTICS
Total number of samples in dataset: 187248
Number of unique patients: 9343
Dataset mode: multimodal

PATIENT BREAKDOWN:
--------------------------------------------------
Patient 5456: 8 modalities
Patient 5806: 4 modalities
Patient 10056: 4 modalities
Patient 5244: 4 modalities
Patient 630: 8 modalities
Patient 10298: 4 modalities
Patient 2080: 10 modalities
Patient 10557: 4 modalities
Patient 5010: 8 modalities
Patient 714: 19 modalities
Patient 3860: 11 modalities
Patient 2136: 4 modalities
Patient 6942: 4 modalities
Patient 6207: 4 modalities
Patient 3650: 18 modalities
Patient 5029: 7 modalities
Patient 7979: 19 modalities
Patient 5047: 2 modalities
Patient 6680: 13 modalities
Patient 10243: 8 modalities
Patient 5450: 3 modalities
Patient 553: 12 modalities
Patient 4116: 4 modalities
Patient 10414: 3 modalities
Patient 10954: 9 modalities
Patient 10443: 4 modalities
Patient 5283: 4 modalities
Patient 4721: 4 modalities
Patient 3247: 8 modalities
Patient 7865: 5 

In [2]:

dataset = CombinedPretrainingV2(
    patch_size=(96, 96, 96),
    data_dir="/home/mg873uh/Projects_kb/data/pretrain_preproc/FOMO60k/",
    mode='multimodal'
)

batch_sampler = TrackedUniquePatientBatchSampler(
    dataset, 
    batch_size=16, 
    drop_last=True,
    shuffle=True
)

dataloader = DataLoader(dataset, batch_sampler=batch_sampler)

UniquePatientBatchSampler initialized:
  Total patients with valid samples: 10387
  Total dataset indices: 162672
  Batch size: 16
  Batches per epoch: 649
  Samples per epoch: 10384


In [1]:
x = dataset.__getitem__(2)
img1 = x['view1']
img2 = x['view2']

import matplotlib.pyplot as plt

images = []
for i in range(260, 270):
    x = dataset[i]  # single call
    img1 = x['view1']
    img2 = x['view2']
    path1 = x['path1'].split('/')[-1]
    path2 = x['path2'].split('/')[-1]
    images.append((img1, img2, path1, path2))

plt.figure(figsize=(20, 4))
for i, (img1, img2, path1, path2) in enumerate(images):
    # Top row (View 1)
    plt.subplot(2, 10, i + 1)
    plt.imshow(img1[0, 0, :, :].cpu().numpy(), cmap='gray')
    plt.title(f"View1 {i+1}\n{path1}", fontsize=8)
    plt.axis('off')
    
    # Bottom row (View 2)
    plt.subplot(2, 10, i + 11)
    plt.imshow(img2[0, 0, :, :].cpu().numpy(), cmap='gray')
    plt.title(f"View2 {i+1}\n{path2}", fontsize=8)
    plt.axis('off')

plt.tight_layout()
plt.show()

NameError: name 'dataset' is not defined